<h4>Lily Grumbach
    <br>M1 Humanités numériques - Université PSL</h4>
<h1><center>Rendu TAIS-TAL </center></h1>
<h2><center>Partie TAL</center></h2>
<h3><center>2/3 : Reconnaissance d'entités nommées sur Doccano</center></h3>

<b><u>PLAN:</u></b>
1) Exploration des données
<br>
**2) NER avec Doccano**

    a) initialisation des données
    b) Exploration de la NER préalable de Spacy
    c) Mise en place de la stratégie pour l'annotation sur Doccano
    d) Entraînement par 5-fold cross validation

3) Application du NER au df

**Enjeux de la partie 2:**
Les deux revues scientifiques étudiées reposent sur trois éléments consitutifs: 
- **Thématiques** : 
    - Hygiène 
    - Pathologies dites "exotiques" ou "coloniales"
    - Liens avec les sciences humaines et sociales 
    - Relations avec les découvertes récentes
- **Terrains spécifiques**  : 
    - Colonies et protectorats français
    - Batiments navals de la marine française
    - Autres territoires hors métropole et possessions françaises
    - Dans certains cas des laboratoires
- **Corps de métier** Population particulière de rédacteurs que sont les membres du corps de santé des colonies et du corps de médecine navale(fait l'objet d'une analyse de réseau non prise en considération dans ce devoir)


**Objectifs de la partie 2:**
- Entraîner une NER sur notre BDD

<br></br>
**L'objectif à long terme** de cet entraînement NER est de :
- Mettre en place une NER qui pourrait être appliquée au full text de chacune des revues une fois les OCR nettoyées. 
- Appliquer cette NER à la revue des Archives de médecine navale et coloniale (1892-1898)
- Alimenter une future base de données relationnelle


**Limites de mon travail:** Il me sera nécessaire de mesurer si elle fonctionne bien à la fois sur le reste de la période étudiée ainsi que sur le Bulletin de la Société de Pathologie exotique


# 0. Initialisation

In [1]:
#Généraux : 
import spacy
import fr_core_news_md
import pandas as pd
import numpy as np
import re


#Importation de mon module 
import Training_Doccano
from Training_Doccano import pandas_to_doccano

#Autres
import pickle
from collections import Counter
from os import path
from pathlib import Path
import glob


In [2]:
#Import des dataframes 
dfAHMC = pd.read_csv("./Notebook_data/input/AHMC_articles-originaux_Termine2.tsv" ,sep='\t')

df_spacyAHMC = dfAHMC.loc[dfAHMC["revue_annee"]<= 1908]
df_spacyAHMC = df_spacyAHMC[["article_titre","revue_annee"]]

dfAMN=pd.read_csv("./Notebook_data/input/AMN-articles-98-08_clean-main.tsv",sep="\t")
dfAMN=dfAMN[:-2] #les deux derniers rows étaient des nan
df_spacyAMN = dfAMN[["article_titre","revue_annee"]]


#print("Nombre d'articles dans chaque revue de 1898 à 1908:",
#       "\nAHMC :",len(df_spacyAHMC),
#       "\nAMN : ",len(df_spacyAMN),
#       "\nTotal à traiter : ",len(df_spacyAHMC)+len(df_spacyAMN))

In [3]:
#On cheche dans un premier temps à avoir une vision générale des annotations du texte et non spécifiquement par article
#Import du full text de chaque revue. 
with open("./Notebook_data/titres_articles_full_AHMC-to-1908.txt") as f:  
    AHMC = f.readlines()
    AHMC_txt = AHMC[0]

with open("./Notebook_data/titres_articles_full_AMN-1898-1908.txt") as f:
    AMN = f.readlines()
    AMN_txt = AMN[0]
    
Comparaison_txt = AMN_txt + AHMC_txt

# 1. Explorations avec le pretrained model de Spacy

In [4]:
#Importation du pretrained model en français
nlp = spacy.load("fr_core_news_md")

docAHMC = nlp(AHMC_txt)
docAMN = nlp(AMN_txt)
docComparaison = nlp(Comparaison_txt)
type(docAHMC)

spacy.tokens.doc.Doc

In [5]:
def show_ents(doc): 
    L=[]
    if doc.ents: 
        for ent in doc.ents: 
            annotations=ent.text+' - ' +str(ent.start_char) +' - '+ str(ent.end_char) +' - '+ent.label_+ ' - '+str(spacy.explain(ent.label_))
            L.append(annotations)
#             print(annotations)
#             print("\n")

In [6]:
# show_ents(docComparaison)

In [7]:
# from spacy import displacy
# displacy.render(docComparaison, style = "ent",jupyter = True)

In [8]:
from Training_Doccano import list_ents_by_label
LOCentities = list_ents_by_label(docComparaison,"LOC")
ORGentities = list_ents_by_label(docComparaison,"ORG")
PERentities = list_ents_by_label(docComparaison,"PER")
MISCentities = list_ents_by_label(docComparaison,"MISC")

#On remarque une grande disparité de tailles 
# print("Taille de chaque label pour l'ensemble des textes:",
#       "\n LOC:",len(LOCentities),
#       "\n PER :",len(PERentities),
#       "\n MISC :",len(MISCentities),
#       "\n ORG :",len(ORGentities))

In [9]:
#Visualiser dans un même tableau l'ensemble des entités récupérées
df_ents_default = pd.DataFrame(
    {"LOC":LOCentities+list(range(9)), #Rééquilibrage pour pouvoir  
     "ORG":ORGentities+list(range(426)),
     "PER":PERentities+list(range(105)),
     "MISC":MISCentities})
df_ents_default.to_csv("./Notebook_data/Training_Doccano/df_ents_default_AHMC&AMN.tsv",sep="\t")
df_ents_default

,LOC,ORG,PER,MISC
0,Dr L. Plazy,Service médical,docteur Quoy,De l'origine toxi
1,Dr Auffret,Septième conférence internationale de la Croix...,M. E. Maurel,Dr Titi
2,Buenos,Dr Valence,Le Hunte,Dr Tatsusaburo Yabé
3,Surra,Dr BelletChirurgie,Dr Richer de Forges,La lutte contre la peste
4,Subsistances,Service de la vaccine,Dr Liffran,Exposition d'hygiène
...,...,...,...,...
514,4,421,100,Dr TitiNote
515,5,422,101,Délivrance de ceintures de flanelle
516,6,423,102,Dr Suard
517,7,424,103,NolletÉtude


<b>Commentaire du nlp par défaut de Spacy</b> :

* `LOC` :
    * Trop de `<GPE>` considérés comme `<LOC>` 
    * "Variété" pris comme `<LOC>`   

* Beaucoup trop de `<MISC>` se référençant tant à des personnes qu'à des lieux,des termes sans aucun sens. 

* Pb annotation comme "Paludisme" en `<ORG>` 

* La NER ne respecte pas bien la tokenisation,notamment dans les cas où il y a un trait d'union.
Ex : "Ssé" -"Mao" au lieu de "Ssé-Mao".


Ces observations justifient bien la nécessité de ne pas s'en tenir au NLP de Spacy et à entraîner notre propre NER.

In [10]:
from spacy.tokenizer import Tokenizer
from spacy.lang.fr import French
tokenizer = Tokenizer(nlp.vocab)

# 2. PREPARATION DES FICHIERS POUR DOCCANO


## 2.1. Création du 4-fold set pour Doccano 

**Rappel:**
Nombre d'articles dans chaque revue de 1898 à 1908: 
<br><u>AHMC</u> : 559 
<br><u>AMN</u> :  638 
<br><b><u>Total à traiter :  1197</u></b>

En prenant un test_set size de 25%, on va donc chercher à avoir un 4-folds cross validation.Nous justifions ici le choix d'une cross-validation en raison de la "sécurité" qu'elle apporte par rapport à un "simple" training/test set. 

**Documentation utilisée** : <br>
Cross-validation: 
https://scikit-learn.org/stable/modules/cross_validation.html


Tuto train_set_split: https://www.youtube.com/watch?v=ZHPwPHkrT4I


ARTICLE À LIRE : https://towardsdatascience.com/complete-guide-to-pythons-cross-validation-with-examples-a9676b5cac12


In [11]:
# Import des librairies scikit-learn 
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_validate, cross_val_score
from sklearn.datasets import load_iris, load_boston
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Import des libraries pour visualiser les datasets
import matplotlib.pyplot as plt
import seaborn as sns
import random

### 2.1.1. Création du 4-folds set

In [12]:
data_set = df_spacyAHMC.article_titre.tolist() + df_spacyAMN.article_titre.tolist()
df_data_set = pd.DataFrame({"articles_AHMC_AMN":data_set})

#on choisit un modèle de 4-fold avec Shuffle car nos données sont pas ordre chronologique et par revue
kf4 = KFold(n_splits=4, shuffle=True, random_state=42)

#Visualisation d'une partie des indexs de la division


### 2.1.2. Création de dictionnaires contenant chaque sous-set:

In [13]:
###ictionnaire avec les indexs de chaque article: 
n=1

dico_sets_index= {}

for train_index, test_index in kf4.split(data_set):
    n_train="train_set"+"#"+str(n)
    n_test="test_set"+"#"+str(n)
    dico_sets_index[n_train]= train_index
    dico_sets_index[n_test]= test_index
    n+=1
    #print("train:",train_index[:15],"\nTest",test_index[:15])
    
# print(dico_sets_index)

In [14]:
###Dictionnaire avec les titres d'article 

dico_sets_articles= {}

n=1
for train_index, test_index in kf4.split(data_set):
    n_train="train_set"+str(n)
    n_test="test_set"+str(n)
    dico_sets_articles[n_train]= np.take(data_set,train_index)
    dico_sets_articles[n_test]= np.take(data_set,test_index)
    n+=1
len(dico_sets_articles)

8

In [15]:
#exportation des textes de train/test sets sous forme de liste :

for key in dico_sets_articles:
    output = "./Notebook_data/Training_Doccano/train-test_sets_articles/"+key
    with open (output,"w") as f:
        for article in dico_sets_articles[key]:
            f.write(article)
            f.write(" \n ")

### 2.1.2. Visualisation du KFold


In [16]:
#Visualisation du 4-fold:
from Training_Doccano import visualize_groups


In [17]:

# for train_index, test_index in kf4.split(df_data_set):
#     X_train = df_data_set.iloc[train_index].loc[:, features]
#     X_test = df_data_set.iloc[test_index].loc[:,features]
#     y_train = df_data_set.iloc[train_index].loc[:,'target']
#     y_test = df_data_set.loc[test_index].loc[:,'target']# visualize_groups(y, groups, 'no groups')

# plt.scatter(x=y_train.index,y=df_data_set.iloc[train_index].loc[:,'target_name'],label ="train")
# plt.scatter(x=y_test.index,y=df_data_set.iloc[test_index].loc[:,'target_name'], label = "test")
# plt.legend()
# plt.show()

### 2.1.3. Conversion des folds en JSONL pour Doccano

#### Transformation de chaque df dans un format compatible avec l'annotation de Doccano 
<br>
Avant d'utiliser Doccano pour corriger les annotations automatiques faites par Spacy, il est nécéssaire de pouvoir les importer dans ce premier. Autrement dit, les annotations contenues dans l'objet Doc de Spacy seront transformées dans un format lisible par Doccano, il s'agit ici du format JSONL, une forme de JSON. Le résultat sera enregistré dans un fichier qui sera dans le même dossier.

<br></br>
Comme j'ai recours à la Kfoldclassification, je dois extraire pour chaque partie du kfold un document JSONL exploitable sur Doccano.

In [18]:
from Training_Doccano import dicoKfolds_to_doccano
dicoKfolds_to_doccano(dico_sets_articles)

###NB: comme la fonction write dans le module est en mode "append", 
### j'ai créé un autre directory "kfolds-fixe" qui est celui de référence 
### pour éviter d'utiliser des documents qui auraient été indentés trop de fois 

File train_set1 created!
File test_set1 created!
File train_set2 created!
File test_set2 created!
File train_set3 created!
File test_set3 created!
File train_set4 created!
File test_set4 created!


## 2.2. Correction des annotations sur Doccano 

1/Lancer l'Environnement virtuel (depuis ~)
* `cd Python-virtual_environment`
* `source ~/Python-virtual_environment/env-virtuel/bin/activate`
=> s'assurer qu'on a bien 'env-virtuel') au début de la ligne de commande

2/Lancer Doccano dans le terminal: 
* Ecrire "`Doccano`" dans le terminal

aller sur : http://localhost:8000/. 

Sources utiles : 

* https://towardsdatascience.com/explorations-in-named-entity-recognition-and-was-eleanor-roosevelt-right-671271117218 

* https://docs.nlpcloud.io/#introduction

* Evaluation:http://www.davidsbatista.net/blog/2018/05/09/Named_Entity_Evaluation/


### 2.2.1. Labels

Labels : 

> * `<LOC>` - Non-GPE locations, mountain ranges, bodies of water
> * `<GPE>` - Geopolitical entity 
> * `<PATH>` - Nom d'affection pathologique (paludisme, béribéri, Peste..) et les substatifs associés (lépreux, pestiféré etc.) 
> * `<ETHNO>` - Reference à une pratique du terrain : "Géographie médicale", "race", "coutume" etc.
> * `<CHEM>` - Les noms de substances chimiques et pharmaceutiques.

Important : 

il y a de nombreux noms d'organisation qui contiennent des noms de `GPE/LOC` ou de `PATH` comme par exemple, "Mission du Baoulé" ou "Mission du paludisme". Etant donné mon axe de recherche, je fais le choix (certes criticable) de faire primer le sens de la `GPE/LOC` / `PATH` sur le nom de l' `ORG ` car c'est souvent le seul indice dans le titre de l'article sur l'affection ou le lieu mentionné. 

<br>
Problème : certains termes qui désignent une pathologie 

Le fichier des labels se trouve dans le working directory. <br>Path : `./DOCCANO/doccano-input/project_labels.json`


# 3. ENTRAÎNEMENT DU NLP 

Chaque sous-partie correspond à une étape du cross validation et comprend les trois étapes suivantes:
- Entrainement du nlp avec les annotations sur le training set
- Validation 
- Evaluation

## 3.1. KFOLD 1

Documents du dataset associés : train_set1.jsonl et test_set1.jsonl

### 3.1.1. entrainement du nlp avec les annotations sur le training set

In [19]:
#Import du JSONL annoté sur Doccano
json_doc_path = "./DOCCANO/doccano-output/train_set1-annotatedd.jsonl"

In [20]:
#Entraînement de ` fr_core_news_md ` avec les annotations stockées dans ` TRAIN_DATA `

from Training_Doccano import train_data
from Training_Doccano import train_nlp

TRAIN_DATA=train_data(json_doc_path)

train_nlp("nlp1",TRAIN_DATA) 
# TRAIN_DATA

#pb de Misaligned entities... signifie que j'ai du annoter de manière peu respectueuse des tokens préalables..
#il n'y en a "que" 4 donc je laisse passer pour cette fois. 

iteration: 0


/home/humanum/anaconda3/lib/python3.8/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "Épidémie de béribéri observée à Poulo-Condore en 1..." with entities "[(32, 46, 'GPE'), (12, 20, 'PATH')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/home/humanum/anaconda3/lib/python3.8/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the text "Fonctionnement de l'Institut Pasteur de Tananarive..." with entities "[(20, 51, 'ORG')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/home/humanum/anaconda3/lib/python3.8/site-packages/spacy/training/iob_utils.py:139: UserWarning: [W030] Some entities could not be aligned in the tex

iteration: 1
iteration: 2
training is finished
Saved model to Notebook_data/Training_Doccano/trained_nlp/nlp1


Je veux aussi voir ce que cela donne sur des articles de la BSPE aussi de 1908 à 1914 (l'enjeu est ici de voir si ça prend bien en considération le vocabulaire plus techniques de la SPE)